In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
dataset=r'Mask Dataset'
imagepath=list(paths.list_images(dataset))

In [3]:
data=[]
labels=[]
for i in imagepath:
    label=i.split(os.path.sep)[-2]
    labels.append(label)
    image=load_img(i,target_size=(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)

In [4]:
data=np.array(data,dtype='float32')
labels=np.array(labels)

In [5]:
labelb=LabelBinarizer()
labels=labelb.fit_transform(labels)
labels=to_categorical(labels)

In [6]:
train_X,test_X,train_Y,test_Y=train_test_split(data,labels,test_size=0.20,random_state=10,stratify=labels)

In [7]:
aug=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')

In [8]:
baseModel=MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

9420800/9406464 [==============================] - 12s 1us/step


In [9]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

model=Model(inputs=baseModel.input,outputs=headModel)

In [10]:
for layer in baseModel.layers:
    layer.trainable=False

In [11]:
learning_rate=0.001
Epochs=20
Batch_size=12

opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

M=model.fit(
    aug.flow(train_X,train_Y,batch_size=Batch_size),
    steps_per_epoch=len(train_X)//Batch_size,
    validation_data=(test_X,test_Y),
    validation_steps=len(test_X//Batch_size),
    epochs=Epochs
)


C:\Users\rarav\AppData\Roaming\Python\Python37\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/20
91/91 [==============================] - 28s 286ms/step - loss: 0.2062 - accuracy: 0.9191 - val_loss: 0.0524 - val_accuracy: 0.9819
Epoch 2/20
91/91 [==============================] - 26s 283ms/step - loss: 0.0679 - accuracy: 0.9743 - val_loss: 0.0526 - val_accuracy: 0.9819
Epoch 3/20
91/91 [==============================] - 26s 283ms/step - loss: 0.0418 - accuracy: 0.9881 - val_loss: 0.0470 - val_accuracy: 0.9819
Epoch 4/20
91/91 [==============================] - 26s 288ms/step - loss: 0.0268 - accuracy: 0.9936 - val_loss: 0.0501 - val_accuracy: 0.9819
Epoch 5/20
91/91 [==============================] - 25s 277ms/step - loss: 0.0388 - accuracy: 0.9862 - val_loss: 0.0371 - val_accuracy: 0.9855
Epoch 6/20
91/91 [==============================] - 26s 289ms/step - loss: 0.0477 - accuracy: 0.9844 - val_loss: 0.0398 - val_accuracy: 0.9855
Epoch 7/20
91/91 [==============================] - 25s 269ms/step - loss: 0.0249 - accuracy: 0.9926 - val_loss: 0.0760 - val_accuracy: 0.9746

In [12]:
predict=model.predict(test_X,batch_size=Batch_size)
predict=np.argmax(predict,axis=1)
print(classification_report(test_Y.argmax(axis=1),predict,target_names=labelb.classes_))

              precision    recall  f1-score   support

   with_mask       1.00      0.99      0.99       138
without_mask       0.99      1.00      0.99       138

    accuracy                           0.99       276
   macro avg       0.99      0.99      0.99       276
weighted avg       0.99      0.99      0.99       276

